In [1]:
!pip install transformers peft torch datasets accelerate bitsandbytes

   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ------------------------- -------------- 6.6/10.4 MB 30.8 MB/s eta 0:00:01
   ---------------------------------------- 10.4/10.4 MB 28.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/75.4 MB ? eta -:--:--
   ----- ---------------------------------- 11.0/75.4 MB 49.1 MB/s eta 0:00:02
   ----------- ---------------------------- 21.8/75.4 MB 50.9 MB/s eta 0:00:02
   ----------------- ---------------------- 33.8/75.4 MB 51.2 MB/s eta 0:00:01
   ----------------------- ---------------- 43.8/75.4 MB 50.7 MB/s eta 0:00:01
   ----------------------------- ---------- 55.3/75.4 MB 51.1 MB/s eta 0:00:01
   ----------------------------------- ---- 66.6/75.4 MB 51.1 MB/s eta 0:00:01
   ---------------------------------------  75.2/75.4 MB 50.5 MB/s eta 0:00:01
   ---------------------------------------  75.2/75.4 MB 50.5 MB/s eta 0:00:01
   ---------------------------------------- 75.4/75.4 MB 39.8 MB/s eta 

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `mytoken` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `mytoken`


In [2]:
import json
from datasets import Dataset

# Load JSON data
def read_in_data(json_file):
    with open(json_file, "r") as f:
        return json.load(f)

# Read and format dataset
JSON_FILE = "prompts_AHandfulOfDust_n1.json"
jdata = read_in_data(JSON_FILE)

dataset = Dataset.from_dict({"text": [d['prompt'] + d['ground_truth'] for d in jdata[5:35]]})

# Split dataset for training & evaluation
dataset = dataset.train_test_split(test_size=0.2)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load model & tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True,  # only use for LORA, only use on GPU
)


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1.
401 Client Error. (Request ID: Root=1-6802ceec-5ed6d6075e59f4b46f27f2d5;7083315b-92e3-4fb0-a4aa-b4055ccd5845)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.1 is restricted. You must have access to it and be authenticated to access it. Please log in.

In [4]:
print(model)

NameError: name 'model' is not defined

In [6]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=4,             # Rank
    lora_alpha=16,   # Scaling factor
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]  # Apply LoRA to attention layers
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 1,703,936 || all params: 7,243,436,032 || trainable%: 0.0235


In [9]:
tokenizer.pad_token = tokenizer.unk_token

def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)
    tokenized["labels"] = tokenized["input_ids"].copy()  # Labels should be the same as input_ids
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])



Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    report_to="none",
    label_names=["labels"], #TODO: check if this actually does anything
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"]
)

trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,2.526900,No log
2,2.247500,No log
3,2.029900,No log


TrainOutput(global_step=36, training_loss=2.1945505142211914, metrics={'train_runtime': 177.8623, 'train_samples_per_second': 0.405, 'train_steps_per_second': 0.202, 'total_flos': 786570563026944.0, 'train_loss': 2.1945505142211914, 'epoch': 3.0})

In [14]:
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_length=500)
    return tokenizer.decode(output[0], skip_special_tokens=True)

for j in jdata[:5]:
  prompt = j['prompt']
  print("===========================================")
  print(prompt)
  print(generate_response(prompt))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Character Info: Mrs. Beaver: A practical, somewhat opportunistic social climber who profits from society's misfortunes. She represents the materialism and opportunism of the time.
John Beaver: A cold and opportunistic social climber who begins an affair with Brenda Last, ultimately contributing to Tony's personal collapse.

Dialogue History: John Beaver: Was anyone hurt?
Instruction: Continue the dialogue for 1 lines.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Character Info: Mrs. Beaver: A practical, somewhat opportunistic social climber who profits from society's misfortunes. She represents the materialism and opportunism of the time.
John Beaver: A cold and opportunistic social climber who begins an affair with Brenda Last, ultimately contributing to Tony's personal collapse.

Dialogue History: John Beaver: Was anyone hurt?
Instruction: Continue the dialogue for 1 lines.
John Beaver: No, but I'm sure they'll be shaken up.

John Beaver: I'm sorry, I didn't mean to cause any trouble.
Instruction: Continue the dialogue for 1 lines.
John Beaver: I just wanted to make sure you were okay.

John Beaver: I'm sorry, I didn't mean to cause any trouble.
Instruction: Continue the dialogue for 1 lines.
John Beaver: I just wanted to make sure you were okay.

John Beaver: I'm sorry, I didn't mean to cause any trouble.
Instruction: Continue the dialogue for 1 lines.
John Beaver: I just wanted to make sure you were okay.

John Beaver: I'm sorry, I didn't 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Character Info: Mrs. Beaver: A practical, somewhat opportunistic social climber who profits from society's misfortunes. She represents the materialism and opportunism of the time.
John Beaver: A cold and opportunistic social climber who begins an affair with Brenda Last, ultimately contributing to Tony's personal collapse.

Dialogue History: John Beaver: Was anyone hurt?
Mrs. Beaver: No one, I am thankful to say, except two housemaids who lost their heads and jumped through a glass roof into the paved court. They were in no danger. The fire never reached the bedrooms, I am afraid. Still, they are bound to need doing up, everything black with smoke and drenched in water and luckily they had that old-fashioned sort of extinguisher that ruins _everything_. One really cannot complain. The chief rooms were _completely_ gutted and everything was insured. Sylvia Newport knows the people. I must get on to them this morning before that ghoul Mrs Shutter snaps them up.
Instruction: Continue the 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Character Info: Mrs. Beaver: A practical, somewhat opportunistic social climber who profits from society's misfortunes. She represents the materialism and opportunism of the time.
John Beaver: A cold and opportunistic social climber who begins an affair with Brenda Last, ultimately contributing to Tony's personal collapse.

Dialogue History: John Beaver: Was anyone hurt?
Mrs. Beaver: No one, I am thankful to say, except two housemaids who lost their heads and jumped through a glass roof into the paved court. They were in no danger. The fire never reached the bedrooms, I am afraid. Still, they are bound to need doing up, everything black with smoke and drenched in water and luckily they had that old-fashioned sort of extinguisher that ruins _everything_. One really cannot complain. The chief rooms were _completely_ gutted and everything was insured. Sylvia Newport knows the people. I must get on to them this morning before that ghoul Mrs Shutter snaps them up.
Mrs. Beaver: Heavens, how 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Character Info: Nanny: A nurturing and steady figure in Tony Last's household, providing a stable presence in the tumultuous events of the family.
Tony Last: The novel's protagonist, a gentleman devoted to his estate, Hetton Abbey. His naive attachment to tradition and his wife, Brenda, leads to his ultimate downfall.
Brenda: Tony Last's wife, who becomes dissatisfied with her marriage and has an affair with John Beaver, ultimately prioritizing her own social standing over her marriage.
Ben: A worker in the Last household, representing the invisible labor that sustains upper-class families.
John Andrew: Tony and Brenda Last's young son, whose tragic death marks the turning point of the novel and accelerates Tony's misfortune.

Dialogue History: Tony Last: Has her ladyship been called yet?
Instruction: Continue the dialogue for 1 lines.
Nanny: No, sir. She's gone to bed.
Character Info: Nanny: A nurturing and steady figure in Tony Last's household, providing a stable presence in the tum

In [ ]:
# Save
model.save_pretrained("./lora_tuned_model")
tokenizer.save_pretrained("./lora_tuned_model")

# Load
# from peft import PeftModel

# base_model = AutoModelForCausalLM.from_pretrained(model_name)
# fine_tuned_model = PeftModel.from_pretrained(base_model, "./lora_tuned_model")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
!zip -r lora_tuned_model.zip ./lora_tuned_model

  adding: fine_tuned_model/ (stored 0%)
  adding: fine_tuned_model/adapter_model.safetensors (deflated 8%)
  adding: fine_tuned_model/special_tokens_map.json (deflated 73%)
  adding: fine_tuned_model/tokenizer_config.json (deflated 68%)
  adding: fine_tuned_model/README.md (deflated 66%)
  adding: fine_tuned_model/tokenizer.model (deflated 55%)
  adding: fine_tuned_model/tokenizer.json (deflated 85%)
  adding: fine_tuned_model/adapter_config.json (deflated 55%)
